In [6]:
# === 1. Import Library ===
import pandas as pd
import numpy as np
import os

# === 2. Konfigurasi File ===
# Pastikan path ini sesuai dengan lokasi file Anda
file_path = r"D:\TEKNIK KOMPUTER\S5\Sistem Pengambilan keputusan\SPK\Artikel\AXCEL_PROJEK_SPK_KELOMPOK_12.xlsx"
sheet_name = " ProsesingSAW"

# Cek file
if not os.path.exists(file_path):
    print(f"ERROR: File tidak ditemukan di: {file_path}")
    exit()

df = pd.read_excel(file_path, sheet_name=sheet_name)

# === 3. Definisi Kriteria ===
kolom_alternatif = "Desa"
kriteria = [
    "IbuHamil_Normal",              # Index 0: False (Cost)
    "Bayi_GiziNormal",              # Index 1: False (Cost)
    "IbuHamil_Periksa",             # Index 2: False (Cost)
    "IbuHamil_TTD",                 # Index 3: True  (Benefit)
    "Anak_Terpantau_TumbuhKembang", # Index 4: True  (Benefit)
    "Anak_GiziBuruk"                # Index 5: True  (Benefit)
]

# === 4. Perhitungan Bobot Menggunakan AHP ===
# Matriks Perbandingan Berpasangan (Sesuai input Anda sebelumnya)
n = len(kriteria)
matriks_perbandingan = np.ones((n, n))

# Input Nilai Perbandingan (Upper Triangle)
matriks_perbandingan[0, 1] = 1      
matriks_perbandingan[0, 2] = 1.5    
matriks_perbandingan[0, 3] = 1.5    
matriks_perbandingan[0, 4] = 0.6    
matriks_perbandingan[0, 5] = 0.6    

matriks_perbandingan[1, 2] = 1.5    
matriks_perbandingan[1, 3] = 1.5    
matriks_perbandingan[1, 4] = 0.6    
matriks_perbandingan[1, 5] = 0.6    

matriks_perbandingan[2, 3] = 1      
matriks_perbandingan[2, 4] = 0.4    
matriks_perbandingan[2, 5] = 0.4    

matriks_perbandingan[3, 4] = 0.4    
matriks_perbandingan[3, 5] = 0.4    

matriks_perbandingan[4, 5] = 1      

# Isi Lower Triangle
for i in range(n):
    for j in range(i+1, n):
        matriks_perbandingan[j, i] = 1 / matriks_perbandingan[i, j]

# Hitung Bobot Prioritas AHP
matriks_normalisasi_ahp = matriks_perbandingan / matriks_perbandingan.sum(axis=0)
bobot_ahp = matriks_normalisasi_ahp.mean(axis=1)

# === 5. Uji Konsistensi AHP (Penting untuk Artikel) ===
weighted_sum = matriks_perbandingan @ bobot_ahp
lambda_max = (weighted_sum / bobot_ahp).mean()
CI = (lambda_max - n) / (n - 1)
RI = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24}.get(n, 1.24) # RI untuk n=6 adalah 1.24
CR = CI / RI

print("="*50)
print(f"Consistency Ratio (CR): {CR:.6f}")
if CR < 0.1:
    print("STATUS: KONSISTEN (Valid untuk Artikel)")
else:
    print("STATUS: TIDAK KONSISTEN (Cek ulang matriks perbandingan)")
print("="*50)

# === 6. Perangkingan Menggunakan SAW (Dengan Bobot AHP) ===

# --- UPDATE PENTING: KONFIGURASI JENIS KRITERIA ---
# False = Cost (Nilai Rendah = Skor Tinggi)
# True  = Benefit (Nilai Tinggi = Skor Tinggi)
jenis_kriteria = [False, False, False, True, True, True]

# Normalisasi Data
normalisasi_df = df[kriteria].copy()
for i, kol in enumerate(kriteria):
    if jenis_kriteria[i]: # Benefit
        normalisasi_df[kol] = df[kol] / df[kol].max()
    else: # Cost
        normalisasi_df[kol] = df[kol].min() / df[kol]

# Hitung Skor Akhir (Normalisasi x Bobot AHP)
df["Skor_Akhir"] = (normalisasi_df * bobot_ahp).sum(axis=1)

# Urutkan Ranking (Ranking 1 = Prioritas Tertinggi untuk Intervensi)
hasil_akhir = df[[kolom_alternatif] + kriteria + ["Skor_Akhir"]].copy()
hasil_akhir = hasil_akhir.sort_values(by="Skor_Akhir", ascending=False).reset_index(drop=True)
hasil_akhir["Ranking"] = hasil_akhir.index + 1

# === 7. Tampilkan Hasil ===
print("\n" + "="*100)
print(f"{'Ranking':<10} {'Nama Desa':<30} {'Skor Akhir':<15}")
print("="*100)
for idx, row in hasil_akhir.head(10).iterrows():
    print(f"{row['Ranking']:<10} {row[kolom_alternatif]:<30} {row['Skor_Akhir']:<15.6f}")
print("-"*100)

# === 8. Simpan Output ===
output_path = os.path.join(os.path.dirname(file_path), "Hasil_Analisis_AHP_SAW.xlsx")
hasil_akhir.to_excel(output_path, index=False)
print(f"\nHasil analisis tersimpan di: {output_path}")

Consistency Ratio (CR): 0.000000
STATUS: KONSISTEN (Valid untuk Artikel)

Ranking    Nama Desa                      Skor Akhir     
1          BONTO MATENE                   0.566197       
2          NISOMBALIA                     0.444679       
3          MARADEKAYA                     0.331847       
4          BARUGA                         0.325509       
5          BONTO MATENE                   0.323396       
6          SIPODECENG                     0.314632       
7          BAJI MANGNGAI                  0.305634       
8          DAMAI                          0.286933       
9          LEKOPANCING                    0.286150       
10         PANCIRO                        0.278951       
----------------------------------------------------------------------------------------------------

Hasil analisis tersimpan di: D:\TEKNIK KOMPUTER\S5\Sistem Pengambilan keputusan\SPK\Artikel\Hasil_Analisis_AHP_SAW.xlsx


In [5]:
# === 1. Import Library ===
import pandas as pd
import numpy as np
import os

# === 2. Baca Data dari Sheet SAW ===
# Update path sesuai permintaan
file_path = r"D:\TEKNIK KOMPUTER\S5\Sistem Pengambilan keputusan\SPK\Artikel\AXCEL_PROJEK_SPK_KELOMPOK_12.xlsx"
sheet_name_saw = " ProsesingSAW"

# Cek apakah file ada
if not os.path.exists(file_path):
    print(f"ERROR: File tidak ditemukan di lokasi: {file_path}")
    exit()

df_saw = pd.read_excel(file_path, sheet_name=sheet_name_saw)

print("="*100)
print("PERHITUNGAN METODE AHP (Analytical Hierarchy Process)")
print("="*100)

# === 3. Ambil Kolom Alternatif dan Kriteria dari SAW ===
kolom_alternatif = "Desa"

kriteria = [
    "IbuHamil_Normal",
    "Bayi_GiziNormal", 
    "IbuHamil_Periksa",
    "IbuHamil_TTD",
    "Anak_Terpantau_TumbuhKembang",
    "Anak_GiziBuruk"
]

n = len(kriteria)

print(f"\nJumlah Alternatif: {len(df_saw)}")
print(f"Jumlah Kriteria: {n}")

# === 4. Input Matriks Perbandingan Berpasangan (SIMETRIS) ===
matriks_perbandingan = np.ones((n, n))

# Isi upper triangle
matriks_perbandingan[0, 1] = 1      
matriks_perbandingan[0, 2] = 1.5    
matriks_perbandingan[0, 3] = 1.5    
matriks_perbandingan[0, 4] = 0.6    
matriks_perbandingan[0, 5] = 0.6    

matriks_perbandingan[1, 2] = 1.5    
matriks_perbandingan[1, 3] = 1.5    
matriks_perbandingan[1, 4] = 0.6    
matriks_perbandingan[1, 5] = 0.6    

matriks_perbandingan[2, 3] = 1      
matriks_perbandingan[2, 4] = 0.4    
matriks_perbandingan[2, 5] = 0.4    

matriks_perbandingan[3, 4] = 0.4    
matriks_perbandingan[3, 5] = 0.4    

matriks_perbandingan[4, 5] = 1      

# Isi lower triangle dengan kebalikannya
for i in range(n):
    for j in range(i+1, n):
        matriks_perbandingan[j, i] = 1 / matriks_perbandingan[i, j]

# === 5. Normalisasi Matriks ===
jumlah_kolom = matriks_perbandingan.sum(axis=0)
matriks_normalisasi = matriks_perbandingan / jumlah_kolom

# === 6. Hitung Bobot Prioritas (Eigen Vector) ===
bobot_ahp = matriks_normalisasi.mean(axis=1)

print("\n" + "="*100)
print("BOBOT PRIORITAS KRITERIA")
print("="*100)
print(f"\n{'Kriteria':<40} {'Bobot AHP':<15}")
print("-"*100)
for i, k in enumerate(kriteria):
    print(f"{k:<40} {bobot_ahp[i]:<15.6f}")
print("-"*100)
print(f"{'TOTAL':<40} {bobot_ahp.sum():<15.6f}")

# === 7. Uji Konsistensi ===
print("\n" + "="*100)
print("UJI KONSISTENSI")
print("="*100)

weighted_sum = matriks_perbandingan @ bobot_ahp
lambda_max = (weighted_sum / bobot_ahp).mean()
CI = (lambda_max - n) / (n - 1)
RI_dict = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
RI = RI_dict.get(n, 1.49)
CR = CI / RI

print(f"\nLambda Max (λ max)        : {lambda_max:.6f}")
print(f"Consistency Index (CI)    : {CI:.6f}")
print(f"Random Index (RI)         : {RI:.6f}")
print(f"Consistency Ratio (CR)    : {CR:.6f}")

if CR < 0.1:
    print("\nSTATUS: KONSISTEN (CR < 0.1)")
else:
    print("\nSTATUS: TIDAK KONSISTEN (CR >= 0.1)")

# === 8. Normalisasi Data Alternatif ===
jenis_kriteria = [True, True, True, True, True, False]
normalisasi_alt = df_saw[kriteria].copy()

for i, kolom in enumerate(kriteria):
    if jenis_kriteria[i]:
        normalisasi_alt[kolom] = df_saw[kolom] / df_saw[kolom].max()
    else:
        normalisasi_alt[kolom] = df_saw[kolom].min() / df_saw[kolom]

# === 9. Hitung Skor AHP ===
df_saw["Skor_AHP"] = (normalisasi_alt * bobot_ahp).sum(axis=1)

hasil_ahp = df_saw[[kolom_alternatif] + kriteria + ["Skor_AHP"]].copy()
hasil_ahp = hasil_ahp.sort_values(by="Skor_AHP", ascending=False).reset_index(drop=True)
hasil_ahp["Ranking"] = hasil_ahp.index + 1

# === 10. Tampilkan Hasil ===
print("\n" + "="*100)
print("HASIL PERANGKINGAN AHP")
print("="*100)
print(f"\n{'Ranking':<10} {'Nama Desa':<30} {'Skor AHP':<15}")
print("="*100)

for idx, row in hasil_ahp.head(20).iterrows():
    print(f"{row['Ranking']:<10} {row[kolom_alternatif]:<30} {row['Skor_AHP']:<15.6f}")
    print("-"*100)

print(f"\nTotal: {len(hasil_ahp)} Desa")

# === 11. Simpan Hasil ===
# Menyimpan hasil di folder yang sama dengan file input
output_folder = os.path.dirname(file_path)
output_path = os.path.join(output_folder, "Hasil_AHP.xlsx")

df_matriks = pd.DataFrame(matriks_perbandingan, columns=kriteria, index=kriteria)
df_normalisasi = pd.DataFrame(matriks_normalisasi, columns=kriteria, index=kriteria)

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df_matriks.to_excel(writer, sheet_name='Matriks Perbandingan')
    df_normalisasi.to_excel(writer, sheet_name='Matriks Normalisasi')
    
    df_bobot = pd.DataFrame({
        'Kriteria': kriteria,
        'Bobot AHP': bobot_ahp
    })
    df_bobot.to_excel(writer, sheet_name='Bobot Prioritas', index=False)
    
    df_konsistensi = pd.DataFrame({
        'Parameter': ['Lambda Max', 'CI', 'RI', 'CR', 'Status'],
        'Nilai': [lambda_max, CI, RI, CR, 'KONSISTEN' if CR < 0.1 else 'TIDAK KONSISTEN']
    })
    df_konsistensi.to_excel(writer, sheet_name='Uji Konsistensi', index=False)
    
    hasil_ahp.to_excel(writer, sheet_name='Hasil Perangkingan AHP', index=False)

print("\n" + "="*100)
print(f"Hasil AHP berhasil disimpan ke: {output_path}")
print("="*100 + "\n")

PERHITUNGAN METODE AHP (Analytical Hierarchy Process)

Jumlah Alternatif: 2267
Jumlah Kriteria: 6

BOBOT PRIORITAS KRITERIA

Kriteria                                 Bobot AHP      
----------------------------------------------------------------------------------------------------
IbuHamil_Normal                          0.150000       
Bayi_GiziNormal                          0.150000       
IbuHamil_Periksa                         0.100000       
IbuHamil_TTD                             0.100000       
Anak_Terpantau_TumbuhKembang             0.250000       
Anak_GiziBuruk                           0.250000       
----------------------------------------------------------------------------------------------------
TOTAL                                    1.000000       

UJI KONSISTENSI

Lambda Max (λ max)        : 6.000000
Consistency Index (CI)    : 0.000000
Random Index (RI)         : 1.240000
Consistency Ratio (CR)    : 0.000000

STATUS: KONSISTEN (CR < 0.1)

HASIL PERANGKINGAN A